### 인스타 데이터 수집

In [71]:
from tqdm import tqdm
import os
hashtag_list = ['일상','daily','instadaily','일상스타그램','일상그램','데일리그램','오늘','데일리','인친','예쁘다그램','바쁜일상','감성','좋아그램','생각스타그램','내멋대로산다','감성일기장','친스타그램','인친스타그램','뷰스타그램','인스타뷰티','beauty','뷰스타','뷰티그램','인생템','메이크업','instamakeup','makeup','selfie','selfcam','ootd','셀스타그램','찍스타그램','셀카','셀피','얼스타그램','셀기꾼','흔녀','흔남','셀피스타그램','얼피','커플','커플그램','럽스타그램','커플스타그램','더럽','러블리','연애중','연애스타그램','데이트','데이트스타그램','커플스탭','컵스타그램','인스타패션','패션','견스타그램','펫스타그램','캣스타그램','냥스타그램','요리스타그램','맛집스타그램','맛스타그램','마스타','맛있다그램','먹스타그램','맛있어','취향저격','instafood','foodstagram','food','맛있다','냠냠쩝쩝','술스타그램','오늘뭐먹지','야식','간식','간식그램','간식스타그램','꿀맛','존맛','존맛탱','핵존맛','쿡스타그램','푸드스타그램','온더테이블','커피스타그램','홈쿡','집밥','가정식','foodie','가정식백반','먹스타그램','먹스타','koreafood','집밥스타그램','맛집탐방','카페스타그램','카페추천','맛집그램','혼술','혼밥','여행','여행스타그램','여행그램','야경스타그램','감성여행','여행중','여행기','여행스탭','떠나요','여행일상','걷고싶은길','여행에미치다','여행사진','어디까지가봣니','스탭','또가고싶다','세계여행','00일차','추억팔이','여행병','여행추억','여행일기','trip','trippin','trippics','tripping','tripstagram','travel','instatravel','travelgram','tourist','tourism','traveling','휴가','휴가스타그램','휴가룩','육아','육아그램','육아스타그램','육아맘그램','맘스타그램','애스타그램','세젤귀','세젤예','인스타키즈','키즈스타그램','키즈그램','키즈패션','육아맘','육아일상','육아헬','육아일기','육아인스타','출산선물','베이비그램','딸바보','아들바보','아들그램','딸그램','아들스타그램','딸스타그램','아들맘','딸맘','줌마스타그램','줌마그램']

for hashtag in tqdm(hashtag_list):
    com = f"instalooter hashtag {hashtag} dataset -d -n 200"
    os.system(com)

100%|██████████████████████████████████████████████████| 165/165 [33:35<00:00, 12.22s/it]


### 수집한 데이터 이름 조회

In [72]:
import pandas as pd
def ext_split(data):
    return data.split(".")[0]
filelist = os.listdir("./dataset")
filelist = list(set(pd.Series(filelist).apply(ext_split)))

### 1차 필터링 - 텍스트에 한글, 영어, 숫자가 있는 피드만 추출

In [73]:
from tqdm import tqdm
import json, re
file_name = []
text_list = []
hashtag_list = []
for filename in tqdm(filelist):
    pattern = re.compile("#([0-9a-zA-Z가-힣]*)")
    with open(f"./dataset/{filename}.json", "r") as st_json:
        
        json_file = json.load(st_json)
        try:
            split_data = json_file['edge_media_to_caption']['edges'][0]['node']['text']
        except:
            continue

        m = pattern.findall(split_data)
        
        if(len(m)==0):
            continue
        
        temp_hashtag = []
        
        
        for unit in m:
            if unit == "":
                continue
            else:
                temp_hashtag.append(unit)
        
        split_data = pattern.sub(',',split_data)
        
        file_name.append(filename)
        text_list.append(split_data)
        hashtag_list.append(temp_hashtag)

100%|█████████████████████████████████████████████| 22282/22282 [02:54<00:00, 127.93it/s]


### 해시태그 추출

In [74]:
hashtag_df = pd.DataFrame(columns=['hashtag'])
for id in tqdm(range(len(hashtag_list))):
    hashtag_df.loc[id] = [hashtag_list[id]]
    
filename_df = pd.DataFrame(file_name)
text_df = pd.DataFrame(text_list)

100%|█████████████████████████████████████████████| 15875/15875 [00:33<00:00, 480.62it/s]


### 이미지이름, 텍스트, 해시태그 병합

In [75]:
df = pd.concat([filename_df,text_df,hashtag_df],axis=1)
df.columns=['Name','Contents','captions']

### 2차 필터링 - 광고제거, 한글을 포함한 게시글만 남기기, csv 저장

In [76]:
def delete_line(data):
    data = data.replace("\n","")
    data = data.replace("\r"," ")
    data = data.replace(",","")
    return data

find_hangul = re.compile('[가-힣]+')
def filter_hangul(data):
    if (len(find_hangul.findall(data)) == 0 or len(data)<10):
        return None
    return data


ban_list = ["수익","회원","타로","사주","종목","BJ","바디프로필","타투","수익","주식","배당","당첨","탈모","쇼핑","재테크","투자","소액","재입고","응모","주부","돈벌기","입양","속눈썹","시술","프로필","카카오톡","카톡","배송","플러스친구","제품설명","링크","profiles","슈퍼개미"]
def del_ad(data):
    for keyword in ban_list:
        if keyword in data:
            return None
    return data

# 불필요한 문자 제거
df.Contents = df.Contents.apply(delete_line)


# 광고 제거
df.Contents = df.Contents.apply(del_ad)
df.drop(df.loc[df.Contents.isnull()].index,axis=0,inplace=True)

# 한글 포함 게시글만 남기기
df.Contents = df.Contents.apply(filter_hangul)
df.drop(df.loc[df.Contents.isnull()].index,axis=0,inplace=True)

# 인덱스 초기화
df = df.reset_index(drop=True)

# type 변환
df.Name = df.Name.astype(str)

# 중복제거
df.loc[df.loc[:,'Contents'].drop_duplicates().index]

# # 저장
df.to_csv("./dataset/image_text_tag.csv",index=False,encoding='utf-8-sig')

df

,Name,Contents,captions
0,2768320131198970077,🍌🥞banana pancakes집에 남은 바나나 해치우기 애매할 때 해먹기 딱 좋은...,"[homemade, koreanfood, simplecooking, homecook..."
1,2768372291294523883,울 딸랑구 감기 걸려서 병원 내원하고 집에오니 늦어서 핼스장 못가고 아파트 헬스장에...,"[몸뚱이개조중, 운동, 운동스타그램, 운동하는남자, 운동하는직장인, 운동하는아빠, ..."
2,2768152909861339732,체력고갈,"[여자템, 결혼, 오오티디, 셀카그램, 연애스타그램, 셀기꾼]"
3,2767608636311416933,내가 가장 좋아하는 아쿠아리움 드디어 다녀왔다🦦❤️....... 뷰티 팔...,"[00, 스냅사진, 아쿠아리움, cafe, daily, dailylook, styl..."
4,2759485501676069416,이쁘게 사진좀 찍어 ꉂ (๑¯ਊ¯)σ,"[인스타패션, 감성, 마스타, 강아지, 육아기록, 팔로잉, 베이비인스타, 여행사진,..."
...,...,...,...
9478,2768365553230553195,🥓 생고기모둠 500g 36.0🥩 꼬들목살 13.0🍖 치마살 13.0🥚 계란찜 3...,"[서신동맛집, 식껍]"
9479,2768281622707833913,비행기 타고 여행가고 싶다~~~지난 블라디보스톡 갈때 찍은사진!!,"[추억팔이, 항공샷, 사진스타그램, 여행가고싶다, 블라디보스톡]"
9480,2766773412619039909,• 싱긋💡𝗣𝗵𝗼𝘁𝗼 - Younginsan Natural Forest,"[travel, 자비스, jarvis]"
9481,2768273354904182936,헬퍼 이모님께서 머리 흘러내리면 안이쁘다고 앞 옆 뒤로 왁스 바르고 시작ㅎㅎ 왁스...,[생후326일]


### 사용하지 않는 파일 삭제, 메타 데이터 삭제

In [77]:
# 불필요 이미지 제거
remove_file = list(set(filelist) - set(df.Name))
for target_file in tqdm(remove_file):
    os.remove(f"./dataset/{target_file}.jpg")
    os.remove(f"./dataset/{target_file}.json")
    
# json 제거
for target_file in tqdm(df.Name):
    os.remove(f"./dataset/{target_file}.json")    

100%|██████████████████████████████████████████████| 9483/9483 [00:03<00:00, 2762.31it/s]
